<a href="https://colab.research.google.com/github/Pratham812002/AI-PLAGIARISM/blob/main/AI_plagiarism_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import re
import zipfile
import gdown
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Corrected link for gdown (If needed)
url = 'https://drive.google.com/uc?id=1FUOHz5qeFajb3TQGDo5jZhYnX_u4Jldb'
gdown.download(url, 'plagiarism-detection-dataset.zip', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1FUOHz5qeFajb3TQGDo5jZhYnX_u4Jldb
To: /content/plagiarism-detection-dataset.zip
100%|██████████| 7.94M/7.94M [00:00<00:00, 29.3MB/s]


'plagiarism-detection-dataset.zip'

In [3]:
# Unzip dataset
with zipfile.ZipFile('plagiarism-detection-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/plagiarism_dataset')


loading the dataset

In [4]:
data = pd.read_csv('/content/plagiarism_dataset/train_snli.txt', delimiter='\t', header=None)
data.columns = ['Sentence1', 'Sentence2', 'Label']

Preprocessing - Droping the missing values

In [5]:
data = data.dropna()

In [6]:
# Sample a smaller subset for faster training (adjust sample size as needed)
data = data.sample(frac=0.2, random_state=42)  # Using 20% of data to speed up training

Preprocessing

In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    return text

data['Sentence1'] = data['Sentence1'].apply(clean_text)
data['Sentence2'] = data['Sentence2'].apply(clean_text)
data['combined_data'] = data['Sentence1'] + " " + data['Sentence2']

In [8]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words='english', max_features=3000)  # Reduced features
X = vectorizer.fit_transform(data['combined_data'])
y = data['Label']


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Train faster model (SGDClassifier)
svm_classifier = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, n_jobs=-1)
svm_classifier.fit(X_train, y_train)

SGDClassifier(n_jobs=-1)

In [11]:
# Predictions
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model trained successfully!")
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

Model trained successfully!
Accuracy: 0.65
              precision    recall  f1-score   support

           0       0.67      0.60      0.63      7357
           1       0.64      0.70      0.67      7338

    accuracy                           0.65     14695
   macro avg       0.65      0.65      0.65     14695
weighted avg       0.65      0.65      0.65     14695



In [12]:
# Plagiarism Prediction Function
def predict_plagiarism(text1, text2):
    text1, text2 = clean_text(text1), clean_text(text2)
    combined_text = text1 + " " + text2
    text_vector = vectorizer.transform([combined_text])
    prediction = svm_classifier.predict(text_vector)[0]

    return "yes" if prediction == 1 else "no"


In [13]:
# User Input
txt1 = input("Enter the first text: ")
txt2 = input("Enter the second text: ")

# Predict Plagiarism
result = predict_plagiarism(txt1, txt2)
print(f"Plagiarism: {result}")

Enter the first text: hello
Enter the second text: hello pratham
Plagiarism: yes


This is a working demonstration of an AI-based plagiarism detection system, trained using the Support Vector Machine (SVM) model. The system evaluates the confidence level of its predictions and calculates accuracy metrics in accordance with the specified requirements.

Thank you,
Cyfuture  